In [ ]:
import pandas as pd
import json
import os
import re
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from scipy.sparse import hstack


# ================== 1. 데이터 로드 및 전처리 ==================
# JSON 파일 읽기
def read_json(folder_paths):
    data = []
    for folder_path in folder_paths:
        for file_name in os.listdir(folder_path):
            if file_name.endswith('.json'):
                with open(os.path.join(folder_path, file_name), 'r') as file:
                    json_data = json.load(file)
                    data.append({
                        "newsTitle": json_data["sourceDataInfo"]["newsTitle"],
                        "newsContent": json_data["sourceDataInfo"]["newsContent"],
                        "clickbaitClass": json_data["sourceDataInfo"]["useType"]
                    })
    return pd.DataFrame(data)

# 텍스트 전처리
def preprocess_text(text, stopwords):
    # 백슬래시 제거
    text = text.replace("\\", "")
    # 중복된 공백 제거
    text = re.sub(r"\s+", " ", text).strip()
    return ' '.join(word for word in text.split() if word not in stopwords)


# 불용어 로드
def load_stopwords(path):
    with open(path, 'r', encoding='utf-8') as file:
        return file.read().splitlines()

# 피처 생성
def add_features(df):
    df['titleLength'] = df['cleanedTitle'].apply(len)
    df['contentLength'] = df['cleanedContent'].apply(len)
    df['title_content_ratio'] = df['titleLength'] / (df['contentLength'] + 1e-6)
    df['contentLexicalDiversity'] = df['cleanedContent'].apply(
        lambda x: len(set(x.split())) / len(x.split()) if len(x.split()) > 0 else 0
    )
    df['specialCharRatio'] = df['combined_text'].apply(lambda x: special_char_ratio_features(x)['special_char_ratio'])
    return df

# TF-IDF 및 수치형 피처 결합
def process_features(df, tfidf_vectorizer=None, scaler=None, train=True):
    if train:
        tfidf_vectorizer = TfidfVectorizer()
        scaler = RobustScaler()
        tfidf_matrix = tfidf_vectorizer.fit_transform(df['cleanedContent'])
        scaled_features = scaler.fit_transform(df[['contentLength', 'title_content_ratio', 'contentLexicalDiversity','specialCharRatio']])
    else:
        tfidf_matrix = tfidf_vectorizer.transform(df['cleanedContent'])
        scaled_features = scaler.transform(df[['contentLength', 'title_content_ratio', 'contentLexicalDiversity','specialCharRatio']])
    combined_features = hstack((tfidf_matrix, scaled_features))
    return combined_features, tfidf_vectorizer, scaler

def special_char_ratio_features(text):
    if not isinstance(text, str) or len(text.strip()) == 0:
        return {'special_char_ratio': 0}
    # 특수문자 비율
    special_chars = re.findall(r'[^\w\s]', text)  # 특수문자
    special_char_ratio = len(special_chars) / len(text)
    return {
        'special_char_ratio': special_char_ratio
    }


# ================== 2. 모델 학습 및 평가 ==================
# 모델 평가 함수
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    roc_auc = auc(fpr, tpr)

    print(f"\n{model.__class__.__name__} Evaluation")
    print('Accuracy:\t', accuracy_score(y_test, y_pred))
    print('Recall:\t\t', recall_score(y_test, y_pred))
    print('Precision:\t', precision_score(y_test, y_pred))
    print('AUC:\t\t', roc_auc)
    print('F1 Score:\t', f1_score(y_test, y_pred))
    plt.plot(fpr, tpr, label=f'AUC = {auc(fpr, tpr):.2f}')
    plt.plot([0, 1], [0, 1], linestyle='--')
    plt.title(f'ROC Curve - {model.__class__.__name__}')
    plt.legend()
    plt.show()

# 하이퍼파라미터 튜닝
def grid_search_tuning(model, param_grid, X_train, y_train):
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    print(f"Best Params: {grid_search.best_params_}, Best F1: {grid_search.best_score_:.4f}")
    return grid_search.best_estimator_

# Validation 평가 함수
def evaluate_on_validation(model, X_validation, y_validation):
    y_val_pred = model.predict(X_validation)
    y_val_proba = model.predict_proba(X_validation)[:, 1]
    fpr, tpr, _ = roc_curve(y_validation, y_val_proba)
    roc_auc = auc(fpr, tpr)

    print(f"Validation Set Evaluation - {model.__class__.__name__}")
    print('Accuracy:\t', accuracy_score(y_test, y_val_pred))
    print('Recall:\t\t', recall_score(y_test, y_val_pred))
    print('Precision:\t', precision_score(y_test, y_val_pred))
    print('AUC:\t\t', roc_auc)
    print('F1 Score:\t', f1_score(y_test, y_val_pred))
    plt.plot(fpr, tpr, label=f'AUC = {auc(fpr, tpr):.2f}')
    plt.plot([0, 1], [0, 1], linestyle='--')
    plt.title(f'Validation ROC Curve - {model.__class__.__name__}')
    plt.legend()
    plt.show()




# ====================== 3. 테스트 데이터 적용 ======================
def predict_new_data(test_data_path, model, tfidf_vectorizer, scaler):
    with open(test_data_path, "r", encoding="utf-8") as file:
        test_data = pd.DataFrame(json.load(file))
    
    test_data['cleanedTitle'] = test_data['title'].apply(lambda x: preprocess_text(x, stopwords))
    test_data['cleanedContent'] = test_data['content'].apply(lambda x: preprocess_text(x, stopwords))
    test_data["combined_text"] = test_data["title"] + " " + test_data["content"]
    test_data = add_features(test_data)

    X_test, _, _ = process_features(test_data, scaler, tfidf_vectorizer,train=False)
    predictions = model.predict_proba(X_test)

    for title, proba in zip(test_data['title'], predictions):
        label = "Clickbait" if proba[1] > proba[0] else "Normal"
        print(f"제목: {title}")
        print(f"예측 결과: {label}, Clickbait 확률: {proba[1]:.2f}, Normal 확률: {proba[0]:.2f}\n")


# ================== 4. 주요 실행 ==================
if __name__ == "__main__":
    # 데이터 로드 및 전처리
    folders = [
        "./Training/02.라벨링데이터/TL_Part1_Clickbait_Auto_SO",
        "./Training/02.라벨링데이터/TL_Part1_Clickbait_Direct_SO",
        "./Training/02.라벨링데이터/TL_Part1_NonClickbait_Auto_SO"
    ]
    stopwords = load_stopwords("stopwords-ko.txt")
    df = read_json(folders)
    df['cleanedTitle'] = df['newsTitle'].apply(lambda x: preprocess_text(x, stopwords))
    df['cleanedContent'] = df['newsContent'].apply(lambda x: preprocess_text(x, stopwords))
    df["combined_text"] = df["newsTitle"] + " " + df["newsContent"]
    df = add_features(df)
    

    
    # 데이터 준비
    X, tfidf_vectorizer, scaler = process_features(df, train=True)
    y = df['clickbaitClass']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Validation 데이터 로드 및 전처리
    validation_folders = [
        "./Validation/02.라벨링데이터/VL_Part1_Clickbait_Auto_SO",
        "./Validation/02.라벨링데이터/VL_Part1_Clickbait_Direct_SO",
        "./Validation/02.라벨링데이터/VL_Part1_NonClickbait_Auto_SO"
    ]
    df_validation = read_json(validation_folders)
    df_validation['cleanedTitle'] = df_validation['newsTitle'].apply(lambda x: preprocess_text(x, stopwords))
    df_validation['cleanedContent'] = df_validation['newsContent'].apply(lambda x: preprocess_text(x, stopwords))
    df_validation["combined_text"] = df_validation["newsTitle"] + " " + df_validation["newsContent"]
    df_validation = add_features(df_validation)
    X_validation, _, _ = process_features(df_validation, tfidf_vectorizer, scaler, train=False)
    y_validation = df_validation['clickbaitClass']

    # 하이퍼파라미터 튜닝 및 최적 모델 선정
    rf_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}
    best_rf = grid_search_tuning(RandomForestClassifier(random_state=42), rf_param_grid, X_train, y_train)

    lr_param_grid = {'C': [1, 10], 'solver': ['lbfgs', 'saga']}
    best_lr = grid_search_tuning(LogisticRegression(max_iter=1000), lr_param_grid, X_train, y_train)

    # 최적 모델 평가 및 시각화
    evaluate_model(best_rf, X_test, y_test)
    evaluate_model(best_lr, X_test, y_test)

    # Validation 데이터 평가
    evaluate_on_validation(best_rf, X_validation, y_validation)
    evaluate_on_validation(best_lr, X_validation, y_validation)



In [10]:

predict_new_data("./model_test_data.json", best_lr, tfidf_vectorizer, scaler)

c:\Users\angel\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(


ValueError: could not convert string to float: "방송통신심의위원회는 JTBC 예능 프로그램 '이혼숙려캠프 새로고침'이 지나치게 적나라한 내용을 방영했다고 지적하며 방송 관계자 의견을 듣기로 했다. 방심위는 17일 서울 양천구 목동 방송회관에서 전체 회의를 열고 음주 상태에서 아내에게 폭언하는 남편의 행동이나 선정적인 내용을 방영한 이혼숙려캠프에 대해 관계자 의견진술을 의결했다. 방심위는 이혼숙려캠프가 객관적 근거 없이 남성의 성욕 등에 일반화해 설명하는 성 역할에 대한 고정관념을 조장할 우려가 있다고 판단했다. 김정수 방심위원은 ‘이혼 사유가 내밀한 문제이긴 집안에서 나눈 대화가 여과 없이 노출되고 있다’며 ‘제재받더라도 시청률이 더 중요하다는 제작진의 안일한 인식이 문제’라고 했다. 류희림 방심위원장은 ‘리얼리티 프로그램에서 너무 적나라한 표현이나 사적인 내용이 나오고 성관계 문제까지 나온다’며 ‘아무리 청소년 이용 불가(19금)라고 해도 지나친 측면이 있다’고 했다. 방심위는 비속어나 차별적 표현을 남발한 지상파 3사 예능에 대해서도 관계자 의견진술을 의결했다. 의결진술 대상은 출연진이 ‘어우씨’, ‘죽여버려’ 등이라고 이를 자막으로 표기한 SBS '런닝맨', ‘지?하네’라는 발언을 묵음 처리해 내보낸 MBC '놀면 뭐 하니'다. '세기가 주목할 요단강 매치'라는 표현이나, ‘숏 다리가 쓸모가 다 있네’라는 말과 자막을 내보낸 KBS '1박2일'이다."